In [ ]:
import glob
import numpy as np
import xarray as xr
import pandas as pd

routelink_subset_file = "../test/input/geo/wrf-hydro-run/DOMAIN/routeLink_subset.nc"

usgs_timeslices_folder = "../test/input/geo/wrf-hydro-run/nudgingTimeSliceObs/"

usgs_file_pattern_filter = "2020-03*.usgsTimeSlice.ncdf"

usgs_files = glob.glob(usgs_timeslices_folder + usgs_file_pattern_filter)

len(usgs_files), len(glob.glob(routelink_subset_file))


In [ ]:
# TEST our Function on a single Datset
ds = xr.open_dataset(usgs_files[0])
# ds.stationId.values.bytes.strip()

stationId_da = map(bytes.strip, ds.stationId.values)
# list(map(bytes.isdigit, stationId_da))
from compose import compose

stationId_da_mask = list(
    map(bytes.isdigit, stationId_da)
)  # Make a mask to remove the blank values
stationId_da_mask = list(
    map(compose(bytes.isdigit, bytes.strip), ds.stationId.values)
)  # Make a mask to remove the blank values
stationId = ds.stationId[stationId_da_mask].values.astype(int)

unique_times = np.unique(ds.time.values)
for i, t in enumerate(unique_times):
    unique_times[i] = t.replace(b"_", b"T")

data_vars = {}
for v in ("discharge", "discharge_quality"):
    data_vars[v] = (["stationId"], ds[v][stationId_da_mask].values)
xr.Dataset(data_vars=data_vars, coords={"stationId": stationId, "time": unique_times})
stationId_da = list(map(bytes.strip, ds.stationId.values))
stationId_da_mask = list(
    map(bytes.isdigit, stationId_da)
)  # Make a mask to remove the blank values


In [ ]:
def preprocess_time_station_index(xd):

    stationId_da_mask = list(
        map(compose(bytes.isdigit, bytes.strip), xd.stationId.values)
    )
    stationId = xd.stationId[stationId_da_mask].values.astype(int)

    unique_times_str = np.unique(xd.time.values).tolist()

    # Three different ways to do the dates
    # 1) as strings
    unique_times = np.array(unique_times_str, dtype="str")

    # 2) as strings, but a different way
    #     unique_times = np.unique(xd.time.values)
    # 3) as dates
    #     for i, t in enumerate(unique_times_str):
    #         unique_times_str[i] = t.replace(b"_",b"T")
    #     unique_times = np.array(unique_times_str,dtype="datetime64")

    data_var_dict = {}
    data_vars = ("discharge", "discharge_quality")
    for v in data_vars:
        data_var_dict[v] = (["stationId"], xd[v].values[stationId_da_mask])
    return xr.Dataset(
        data_vars=data_var_dict, coords={"stationId": stationId, "time": unique_times}
    )


# return xr.Dataset(data_vars=data_vars{'discharge': (['stationId'], xd.discharge.values), 'discharge_quality': (['stationId'], xd.discharge_quality.values)},
# coords={'stationId': stationId, 'time': np.unique(xd.time.values)})


In [ ]:
#%%timeit
#%%prun
# How long to preprocess one dataset?
preprocess_time_station_index(ds)


In [ ]:
#%%time
# %%prun
xr.open_mfdataset(
    usgs_files[0 : len(usgs_files) // 1],
    preprocess=preprocess_time_station_index,
    combine="by_coords",
    # data_vars="minimal",
    # coords="minimal",
    # compat="override",
    # chunks= {'time':481,'stationId':1} #3869*481,
    parallel=True,
)


In [ ]:
#%%time
# %%prun
# This version appears to be the fastest
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    paths = sorted(glob.glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined


# you might also use indexing operations like .sel to subset datasets
# combined = read_netcdfs('/all/my/files/*.nc', dim='time',
#                         transform_func=lambda ds: ds.mean())

with read_netcdfs(
    usgs_timeslices_folder + usgs_file_pattern_filter,
    "time",
    preprocess_time_station_index,
) as ds2:
    df2 = pd.DataFrame(
        ds2["discharge"].values.T,
        index=ds2["stationId"].values,
        columns=ds2.time.values,
    )

df2


In [ ]:
#%%time
with xr.open_mfdataset(
    usgs_files, preprocess=preprocess_time_station_index, combine="by_coords"
) as ds2:
    df3 = pd.DataFrame(
        ds2["discharge"].values.T,
        index=ds2["stationId"].values,
        columns=ds2.time.values,
    )

df3


In [ ]:
ds_t = xr.open_dataset(usgs_files[0])
ds_t


In [ ]:
xs = ds_t.set_coords(["stationId", "time"])
xs = xs.set_index(stationIdInd="stationId")
# xr.Dataset(data_vars={"discharge": da_discharge})
xs


In [ ]:
# Open the Route-Link File to convert the indexes
with xr.open_dataset(routelink_subset_file) as ds:
    gage_list = list(map(bytes.strip, ds.gages.values))
    gage_mask = list(map(bytes.isdigit, gage_list))

    gage_da = ds.gages[gage_mask].values.astype(int)

    data_var_dict = {}
    data_vars = ("link", "to", "ascendingIndex")
    for v in data_vars:
        data_var_dict[v] = (["gages"], ds[v].values[gage_mask])
    ds = xr.Dataset(data_vars=data_var_dict, coords={"gages": gage_da})
df = ds.to_dataframe()
df


In [ ]:
usgs_df = df.join(df2)
usgs_df = usgs_df.reset_index()
usgs_df = usgs_df.rename(columns={"index": "gages"})
usgs_df = usgs_df.set_index("link")
usgs_df = usgs_df.drop(["gages", "ascendingIndex", "to"], axis=1)
usgs_df


In [ ]:
columns_list = usgs_df.columns
# columns_list


In [ ]:
for i in range(0, (len(columns_list) * 3) - 12, 12):
    original_string = usgs_df.columns[i]
    original_string_shortened = original_string[:-5]
    temp_name1 = original_string_shortened + str("05:00")
    temp_name2 = original_string_shortened + str("10:00")
    temp_name3 = original_string_shortened + str("20:00")
    temp_name4 = original_string_shortened + str("25:00")
    temp_name5 = original_string_shortened + str("35:00")
    temp_name6 = original_string_shortened + str("40:00")
    temp_name7 = original_string_shortened + str("50:00")
    temp_name8 = original_string_shortened + str("55:00")
    usgs_df.insert(i + 1, temp_name1, np.nan)
    usgs_df.insert(i + 2, temp_name2, np.nan)
    usgs_df.insert(i + 4, temp_name3, np.nan)
    usgs_df.insert(i + 5, temp_name4, np.nan)
    usgs_df.insert(i + 7, temp_name5, np.nan)
    usgs_df.insert(i + 8, temp_name6, np.nan)
    usgs_df.insert(i + 10, temp_name7, np.nan)
    usgs_df.insert(i + 11, temp_name8, np.nan)

usgs_df = usgs_df.interpolate(method="linear", axis=1)
usgs_df
# get_usgs_from_wrf_hydro(routelink_subset_file,usgs_timeslices_folder)


In [ ]:
from compose import compose


def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        with xr.open_dataset(path) as ds:
            if transform_func is not None:
                ds = transform_func(ds)
            ds.load()
            return ds

    paths = sorted(glob.glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined


def preprocess_time_station_index(xd):
    stationId_da_mask = list(
        map(compose(bytes.isdigit, bytes.strip), xd.stationId.values)
    )
    stationId = xd.stationId[stationId_da_mask].values.astype(int)

    unique_times_str = np.unique(xd.time.values).tolist()

    unique_times = np.array(unique_times_str, dtype="str")

    data_var_dict = {}
    data_vars = ("discharge", "discharge_quality")
    for v in data_vars:
        data_var_dict[v] = (["stationId"], xd[v].values[stationId_da_mask])
    return xr.Dataset(
        data_vars=data_var_dict, coords={"stationId": stationId, "time": unique_times}
    )


# you might also use indexing operations like .sel to subset datasets
# combined = read_netcdfs('/all/my/files/*.nc', dim='time',
#                         transform_func=lambda ds: ds.mean())

with read_netcdfs(
    usgs_timeslices_folder + usgs_file_pattern_filter,
    "time",
    preprocess_time_station_index,
) as ds2:
    df2 = pd.DataFrame(
        ds2["discharge"].values.T,
        index=ds2["stationId"].values,
        columns=ds2.time.values,
    )

with xr.open_dataset(routelink_subset_file) as ds:
    gage_list = list(map(bytes.strip, ds.gages.values))
    gage_mask = list(map(bytes.isdigit, gage_list))

    gage_da = ds.gages[gage_mask].values.astype(int)

    data_var_dict = {}
    data_vars = ("link", "to", "ascendingIndex")
    for v in data_vars:
        data_var_dict[v] = (["gages"], ds[v].values[gage_mask])
    ds = xr.Dataset(data_vars=data_var_dict, coords={"gages": gage_da})
df = ds.to_dataframe()

usgs_df = df.join(df2)
usgs_df = usgs_df.reset_index()
usgs_df = usgs_df.rename(columns={"index": "gages"})
usgs_df = usgs_df.set_index("link")
usgs_df = usgs_df.drop(["gages", "ascendingIndex", "to"], axis=1)
columns_list = usgs_df.columns

for i in range(0, (len(columns_list) * 3) - 12, 12):
    original_string = usgs_df.columns[i]
    original_string_shortened = original_string[:-5]
    temp_name1 = original_string_shortened + str("05:00")
    temp_name2 = original_string_shortened + str("10:00")
    temp_name3 = original_string_shortened + str("20:00")
    temp_name4 = original_string_shortened + str("25:00")
    temp_name5 = original_string_shortened + str("35:00")
    temp_name6 = original_string_shortened + str("40:00")
    temp_name7 = original_string_shortened + str("50:00")
    temp_name8 = original_string_shortened + str("55:00")
    usgs_df.insert(i + 1, temp_name1, np.nan)
    usgs_df.insert(i + 2, temp_name2, np.nan)
    usgs_df.insert(i + 4, temp_name3, np.nan)
    usgs_df.insert(i + 5, temp_name4, np.nan)
    usgs_df.insert(i + 7, temp_name5, np.nan)
    usgs_df.insert(i + 8, temp_name6, np.nan)
    usgs_df.insert(i + 10, temp_name7, np.nan)
    usgs_df.insert(i + 11, temp_name8, np.nan)

usgs_df = usgs_df.interpolate(method="linear", axis=1)
usgs_df
